In [ ]:
import json
from urllib.parse import urlencode
import requests
import random
from bs4 import BeautifulSoup
from lxml import etree
import re
import os
from io import BytesIO
from PIL import Image

In [ ]:
!pip install tldextract
from tldextract import extract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 1.8 MB/s 


In [ ]:
import signal
from contextlib import contextmanager

class TimeoutException(Exception): pass

@contextmanager

def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

In [ ]:
def random_headers():
    headers = {
        'User-Agent': random.choice(
            [
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36',
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0',
                'Mozilla/5.0 (Windows NT 10.0; Win 64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36',
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36',
            ]
        )          
    }
    return headers

In [ ]:
def get_google_results(query, num_pages):

    logs = {}

    domain = 'com'
    payload = {
        'q': query, 
        'uule': 'w+CAIQICIGTE9ORE9O'
    }

    links = [] 

    for count in range(0, num_pages * 10, 10):
        payload['start'] = count # One page is equal to 10 google results.

        params = urlencode(payload)
        url = f'https://www.google.{domain}/search?{params}'

        # Scrape.
        response = requests.get(url=url, headers=random_headers())
        soup = BeautifulSoup(response.content, 'html.parser')
        
        unwanted_list = ['google', 'search', '#', 'facebook', 'instagram']

        for ref in set(soup.findAll('a')):
                href  = ref.get('href')
                if type(href) == str:
                    good_link = True
                    for unwanted in unwanted_list:
                        if unwanted in href:
                            good_link = False
                            dict_append_or_create(href, f'Filtered: On {unwanted}', logs)
                    
                    if good_link and 'https' in href:
                        links.append(href)
                    else: 
                        if good_link:
                            dict_append_or_create(href, 'Filtered: https', logs)
                            
    return links, logs

In [ ]:
def url_rank(url):
    rank = 0
    if 'about' not in url.lower():
        rank += 200
    rank += (len(url))
    return rank

In [ ]:
def get_domain(url):
    tsd, td, tsu = extract(url)
    if len(tsd) > 0:
        tsd += '.'
    domain_url = f'https://{tsd}.{td}.{tsu}'
    return url

In [ ]:
tim = {}
tim[2] =['tim']
type(tim[2])

list

In [ ]:
def dict_append_or_create(key, value, d):
    if key not in d.keys():
        d[key] = [value]
    else:
        d[key].append(value)

In [ ]:
def find_about(url):

    logs = {}

    tsd, td, tsu = extract(url)
    domain_url = get_domain(url)

    response = requests.get(url=domain_url, headers=random_headers())
    soup = BeautifulSoup(response.content, 'html.parser')

    possible_links = []

    for ref in set(soup.findAll('a')):
        href  = ref.get('href')
        if type(href) == str:
            if domain_url in href and 'about' in href.lower():
                possible_links.append(href)
            else:
                dict_append_or_create(url, f'Filtered: {href} No about or domain ', logs)

    possible_links.sort(key=url_rank)
                
    if len(possible_links) > 0:
        
        dict_append_or_create(url, f'Found: {possible_links[0]}', logs)
        return possible_links[0], logs
    
    return False, logs

In [ ]:
def google_search_about(url):

    logs = {}

    tsd, td, tsu = extract(url)
    domain_url = get_domain(url)

    domain_url_trimmed = domain_url.replace('https://', '')
    domain_url_trimmed = domain_url_trimmed.replace('http://', '')
    
    query = f'about us {domain_url_trimmed}'
    dict_append_or_create(url, f'Query: {query}', logs)
    
    domain = 'com'
    payload = {
        'q': query, 
        'uule': 'w+CAIQICIGTE9ORE9O'
    }

    params = urlencode(payload)
    search_url = f'https://www.google.{domain}/search?{params}'


    response = requests.get(url=search_url, headers=random_headers())
    soup = BeautifulSoup(response.content, 'html.parser')

    unwanted_list = ['google', 'search', '#', 'facebook', 'instagram']

    possible_links = []

    for ref in set(soup.findAll('a')):
        href  = ref.get('href')
        if type(href) == str:
            good_link = True
            for unwanted in unwanted_list:
                if unwanted in href:
                    good_link = False
                    dict_append_or_create(url, f'Filtered: {href} on {unwanted}', logs)

            if good_link and domain_url in href and 'about' in href.lower():
                possible_links.append(href)
            else:
                dict_append_or_create(url, f'Filtered: {href} on domain/about', logs)
    
    possible_links.sort(key=url_rank)
                
    if len(possible_links) > 0:
        dict_append_or_create(url, f'Found: {possible_links[0]} on via google', logs)
        return possible_links[0], logs
    
    dict_append_or_create(url, f'None found:', logs)
    return False, logs

In [ ]:
def get_image_urls(url):
    
    logs = {}

    response = requests.get(url, headers=random_headers())

    soup = BeautifulSoup(response.text, 'html.parser')
    img_tags = soup.find_all('img')

    dict_append_or_create(url, f'{len(img_tags)} found', logs)

    img_urls = []
    unwanted_list = ['google', 'icon', 'logo', 'banner', 'facebook']

    for img in img_tags:
        source_found = True
        try:
            img_url = img["data-srcset"]
        except:
            try:
                img_url = img["data-src"]
            except:
                try:
                    img_url = img["src"]
                except:
                    try:
                        img_url = img["data-lazy-src"]
                    except:
                        try: 
                            img_url = img["data-fallback-src"]
                        except:
                            try: 
                                img_url = re.search(r'\/\S+?(png|jpg|gif)', str(img)).group(0)
                            except:
                                source_found = False
                                dict_append_or_create(url, f'No soucre found for: {img}', logs)
        
        if source_found:
            if 'http' not in img_url:
                img_url = get_domain(url) + img_url
                dict_append_or_create(url, f'Relative path fixed for {img}', logs)
            
            good_link = True
            for unwanted in unwanted_list:
                if unwanted in img_url:
                    good_link = False
                    dict_append_or_create(url,f'Filtered {img_url} on {unwanted}', logs)
            
            if good_link:
                re_search = re.search(r'https:\/\/\S+?(png|jpg|gif)', img_url)
                if re_search and re_search.group(0) not in img_urls:
                    img_urls.append(re_search.group(0))
                    dict_append_or_create(url,f'{re_search.group(0)} found for {img}', logs)
            else:
                dict_append_or_create(url,f'Filtered {img} on no re match or duplicate', logs)
    return img_urls, logs

In [ ]:
def save_from_url_list(img_urls, save_dir, threshold_dim):

    count = 1
    filename_dict = {}

    logs = {}

    for url in img_urls:
        
        filename = str(count).zfill(5)
        filename_dict[filename] = url

        try:
            response = requests.get(url, headers=random_headers())
            response_content = response.content
            try:
                response_content = str(response_content, 'utf-8')
                logs.append(f'unknown try success for {url}')
            except UnicodeDecodeError:
                i = Image.open(BytesIO(response_content))
                width, height = i.size
                
                if width > threshold_dim and height > threshold_dim:
                    with open(f"{save_dir}/{filename}.jpg", "wb+") as f:
                        f.write(response_content)
                    count += 1
                    logs.append(f'Write attempted for {url} as {filename}')
                else:
                    logs.append(f'Undersize: {url}')
        except:
            logs.append(f'Failed to get content: {url}')
    
    return filename_dict, logs

In [ ]:
results

{'Fail: Undable to get content': [], 'Fail: Undersize': []}

In [ ]:
query = 'london plumbers uk'
num_pages = 1

search_results, search_logs = get_google_results(query, num_pages)

In [ ]:
search_results

['https://london-plumber.co.uk/services/',
 'https://www.trustatrader.com/plumbers-in-london',
 'https://www.pimlicoplumbers.com/',
 'https://www.aspect.co.uk/trades/plumbing-and-cold-water/',
 'https://www.trustatrader.com/plumbers-in-london',
 'https://plumbforcedirect.co.uk/',
 'https://my-plumber.co.uk/plumbing/',
 'https://my-plumber.co.uk/prices/',
 'https://london-plumber.co.uk/',
 'https://my-plumber.co.uk/areas/',
 'https://london-plumber.co.uk/rates/',
 'https://my-plumber.co.uk/fantastic-membership-club/',
 'https://www.aspect.co.uk/trades/plumbing-and-cold-water/',
 'https://www.homecureplumbers.co.uk/',
 'https://my-plumber.co.uk/',
 'https://london-plumber.co.uk/locations/',
 'https://london-plumber.co.uk/',
 'https://london-plumber.co.uk/knowledge-base/',
 'https://plumbforcedirect.co.uk/',
 'https://www.houzz.com/professionals/plumbing-contractors/c/London--UK',
 'https://www.homecureplumbers.co.uk/',
 'https://www.tailoredplumbing.co.uk/']

In [ ]:
about_results = []
about_logs = {}

for url in search_results:
    domain_results = find_about(url)
    about_logs.update(domain_results[1])
    if domain_results[0]:
        about_results.append(domain_results[0])
    else:
        google_results = google_search_about(url)
        about_logs.update(google_results[1])
        if google_results[0]:
            about_results.append(google_results[0])

In [ ]:
about_results

['https://www.pimlicoplumbers.com/about-us/',
 'https://plumbforcedirect.co.uk/about-us.html',
 'https://www.homecureplumbers.co.uk/about-us/',
 'https://my-plumber.co.uk/about-us/',
 'https://plumbforcedirect.co.uk/about-us.html',
 'https://www.homecureplumbers.co.uk/about-us/',
 'https://www.tailoredplumbing.co.uk/about/']

In [ ]:
all_results = list(set(search_results + about_results))

images_logs = {}
images_urls = {}

for url in all_results:

    get_images_reults = get_image_urls(url)
    
    images_urls[url] = get_images_reults[0]
    images_logs.update(get_images_reults[1])

In [ ]:
images_urls

{'https://www.homecureplumbers.co.uk/about-us/': ['https://www.homecureplumbers.co.uk/wp-content/uploads/2020/02/boilerrepairs-1-800x469.jpg',
  'https://www.homecureplumbers.co.uk/wp-content/uploads/2020/02/boilerservice-1-800x469.jpg',
  'https://www.homecureplumbers.co.uk/wp-content/uploads/2020/02/boilerinstall-1-800x469.jpg',
  'https://www.homecureplumbers.co.uk/wp-content/uploads/2020/08/what-makes-us-different@2x-800x469.jpg',
  'https://www.homecureplumbers.co.uk/wp-content/uploads/2020/02/plumbing-1-800x469.jpg',
  'https://www.homecureplumbers.co.uk/wp-content/uploads/2020/02/emergencyplumber-800x469.jpg',
  'https://www.homecureplumbers.co.uk/wp-content/uploads/2020/02/drainage-2-800x469.jpg',
  'https://www.homecureplumbers.co.uk/wp-content/uploads/2020/02/IMG_8133-800x469.jpg',
  'https://www.homecureplumbers.co.uk/wp-content/uploads/2020/02/engineer-800x469.jpg',
  'https://www.homecureplumbers.co.uk/wp-content/uploads/2020/02/powerflush-1-800x469.jpg',
  'https://www.ho

21